In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [948 k

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-05 21:16:47--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2022-10-05 21:16:47 (11.1 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...|2015-08-31 00:00:00|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser

In [ ]:
ratings_df = df.select(["vine", "verified_purchase", "star_rating", "total_votes", "helpful_votes"])
ratings_df.show()

+----+-----------------+-----------+-----------+-------------+
|vine|verified_purchase|star_rating|total_votes|helpful_votes|
+----+-----------------+-----------+-----------+-------------+
|   N|                N|          3|          1|            0|
|   N|                Y|          5|          0|            0|
|   N|                Y|          3|          1|            0|
|   N|                Y|          5|          0|            0|
|   N|                Y|          5|          0|            0|
|   N|                N|          5|          0|            0|
|   N|                Y|          2|          4|            3|
|   N|                Y|          5|          0|            0|
|   N|                Y|          5|          0|            0|
|   N|                N|          5|          1|            1|
|   N|                Y|          5|          0|            0|
|   N|                Y|          4|          0|            0|
|   N|                Y|          3|          0|       

In [ ]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the 
# total_votes count is equal to or greater than 20
votes_df = ratings_df.filter(ratings_df["total_votes"] >= 20)

In [ ]:
# Filter the new DataFrame or table created in Step 1
filtered_votes_df = votes_df.filter(votes_df["helpful_votes"]/votes_df["total_votes"] >= 0.5)
filtered_votes_df.show()

+----+-----------------+-----------+-----------+-------------+
|vine|verified_purchase|star_rating|total_votes|helpful_votes|
+----+-----------------+-----------+-----------+-------------+
|   N|                N|          5|         61|           47|
|   N|                Y|          5|         23|           21|
|   N|                Y|          1|         38|           37|
|   N|                N|          5|         37|           33|
|   N|                Y|          3|         24|           23|
|   N|                Y|          5|         37|           34|
|   N|                N|          5|         23|           22|
|   N|                Y|          5|         37|           37|
|   N|                Y|          5|         20|           19|
|   N|                Y|          2|         22|           21|
|   N|                Y|          5|         27|           27|
|   N|                Y|          3|         85|           72|
|   N|                N|          5|         24|       

In [ ]:
# Filter the DataFrame or table created in Step 2
from pyspark.sql.functions import col, avg
paid_df = filtered_votes_df.filter(filtered_votes_df['vine']== 'Y')
paid_df.show()

+----+-----------------+-----------+-----------+-------------+
|vine|verified_purchase|star_rating|total_votes|helpful_votes|
+----+-----------------+-----------+-----------+-------------+
|   Y|                N|          4|         23|           20|
|   Y|                N|          5|         32|           26|
|   Y|                N|          5|         35|           29|
|   Y|                N|          5|        161|          146|
|   Y|                N|          5|         59|           55|
|   Y|                N|          4|         25|           19|
|   Y|                N|          5|         25|           22|
|   Y|                N|          5|        102|           87|
|   Y|                N|          4|         33|           28|
|   Y|                N|          2|         37|           26|
|   Y|                N|          3|         22|           20|
|   Y|                N|          3|         50|           46|
|   Y|                N|          5|         25|       

In [ ]:
# Repeat Step 3, but this time retrieve all the rows where the review was not 
# part of the Vine program (unpaid
unpaid_df = filtered_votes_df.filter(filtered_votes_df['vine']== 'N')
unpaid_df.show()

+----+-----------------+-----------+-----------+-------------+
|vine|verified_purchase|star_rating|total_votes|helpful_votes|
+----+-----------------+-----------+-----------+-------------+
|   N|                N|          5|         61|           47|
|   N|                Y|          5|         23|           21|
|   N|                Y|          1|         38|           37|
|   N|                N|          5|         37|           33|
|   N|                Y|          3|         24|           23|
|   N|                Y|          5|         37|           34|
|   N|                N|          5|         23|           22|
|   N|                Y|          5|         37|           37|
|   N|                Y|          5|         20|           19|
|   N|                Y|          2|         22|           21|
|   N|                Y|          5|         27|           27|
|   N|                Y|          3|         85|           72|
|   N|                N|          5|         24|       

Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

In [ ]:
total_paid_vine_reviews = paid_df.count()
total_paid_5star_vine_reviews = paid_df[paid_df['star_rating']== 5].count()
percentage_of_paid_5star_vine_reviews = float(total_paid_5star_vine_reviews) / float(total_paid_vine_reviews)
print(percentage_of_paid_5star_vine_reviews)

0.5666666666666667


In [ ]:
total_unpaid_nonvine_reviews = unpaid_df.count()
total_unpaid_5star_nonvine_reviews = unpaid_df[unpaid_df['star_rating']== 5].count()
percentage_of_unpaid_5star_nonvine_reviews = float(total_unpaid_5star_nonvine_reviews) / float(total_unpaid_nonvine_reviews)
print(percentage_of_unpaid_5star_nonvine_reviews)

0.5672445948746287


In [ ]:
print("Total number of reviews for paid/Vine members:") 
print(total_paid_vine_reviews)
print("")
print("Total number of 5-star reviews for paid/Vine members:")
print(total_paid_5star_vine_reviews)
print("")
print("Percentage of 5-star reviews for paid/Vine members:")
print(percentage_of_paid_5star_vine_reviews)

Total number of reviews for paid/Vine members:
60

Total number of 5-star reviews for paid/Vine members:
34

Percentage of 5-star reviews for paid/Vine members:
0.5666666666666667


In [ ]:
print("Total number of reviews for unpaid/non-Vine members:") 
print(total_unpaid_nonvine_reviews)
print("")
print("Total number of 5-star reviews for unpaid/non-Vine members:")
print(total_unpaid_5star_nonvine_reviews)
print("")
print("Percentage of 5-star reviews for unpaid/non-Vine members:")
print(percentage_of_unpaid_5star_nonvine_reviews)

Total number of reviews for unpaid/non-Vine members:
14477

Total number of 5-star reviews for unpaid/non-Vine members:
8212

Percentage of 5-star reviews for unpaid/non-Vine members:
0.5672445948746287
